In [1]:
from flask import Flask, request, jsonify
import os
import logging
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
import random
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever()
docs = retriever.invoke("TOKYO GHOUL")
print(docs[0].page_content[:400])

# Load environment variables from a .env file.
load_dotenv()

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COGNITIVE_SEARCH_URL = os.getenv("COGNITIVE_SEARCH_URL")
COGNITIVE_SEARCH_ADMIN_KEY = os.getenv("COGNITIVE_SEARCH_ADMIN_KEY")

app = Flask(__name__)


@tool("add")
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


@tool("subtract")
def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b


@tool("generate_random_number")
def generate_random_number(min: int, max: int) -> int:
    """Generate a random number between min and max.
    Args:
        min (int): The minimum value of the random number.
        max (int): The maximum value of the random number.
    """
    return random.randint(min, max)

Tokyo Ghoul (Japanese: 東京喰種（トーキョーグール）, Hepburn: Tōkyō Gūru) is a Japanese dark fantasy manga series written and illustrated by Sui Ishida. It was serialized in Shueisha's seinen manga magazine Weekly Young Jump from September 2011 to September 2014, with its chapters collected in 14 tankōbon volumes. The story is set in an alternate version of Tokyo where humans coexist with ghouls, beings who loo


In [3]:
vector_store_address: str = COGNITIVE_SEARCH_URL
vector_store_password: str = COGNITIVE_SEARCH_ADMIN_KEY

embeddings_model: str = "text-embedding-ada-002"

openai_api_version: str = "2023-05-15"
# Option 1: Use OpenAIEmbeddings with OpenAI account
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY, openai_api_version=openai_api_version, model=embeddings_model
)

index_name: str = "stockripper-documents"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

## test vector search by adding some documents
#from langchain_community.document_loaders import TextLoader
#from langchain_text_splitters import CharacterTextSplitter
#
## load all documents in the documents folder
#for file in os.listdir("documents"):
#    if file.endswith(".txt"):
#        loader = TextLoader(f"documents/{file}", encoding="utf-8")
#        documents = loader.load()
#        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#        docs = text_splitter.split_documents(documents)
#        vector_store.add_documents(documents=docs)


# Perform a similarity search
docs = vector_store.similarity_search(
    query="What is an aggressive investment strategy?",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

6. Short-term
Many of the investment strategies discussed so far — such as dollar-cost averaging and value investing — may be best suited for investors with long-term goals. But if you need to use the money in the account within the next one to five years, those investment strategies may not be appropriate for you. 

Short-term investing tends to be conservative. Rather than investing in stocks, short-term investors choose investments that are less risky, such as a mix of bonds, certificates of deposit (CDs), high-yield savings accounts and money market accounts. The returns are often lower than you’d get with the stock market, but there is less risk. 

[Important: Short-term investing is very different from day trading, an investment strategy that involves rapidly buying and selling stocks, often within the same day or even within a few hours. Day trading is highly speculative, and incredibly risky.]


In [ ]:
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain.memory import VectorStoreRetrieverMemory
from langchain.vectorstores.base import VectorStoreRetriever

# Define the vector store retriever for memory
memory_vector_store = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name="agent-memory",
    embedding_function=embeddings.embed_query,
)

# Create a retriever from the vector store
memory_retriever = VectorStoreRetriever(vectorstore=memory_vector_store)

# Define the memory object using the retriever
memory = VectorStoreRetrieverMemory(
    retriever=memory_retriever,
    memory_key="history",  # This key will be used when referencing memory in prompts
    input_key="input",     # The input key to correlate user prompts
    return_docs=False      # To avoid returning entire documents, useful for simplicity
)



In [96]:
import uuid
from langchain_community.vectorstores.azuresearch import Document

# Modify the save_to_memory function to create a Document instance correctly
def save_to_memory(session_id, role, content):
    # Use a proper Document instance with an ID and page_content field
    document = Document(
        id=str(uuid.uuid4()),  # Generate a unique ID for each document
        page_content=content,  # Updated to use 'page_content'
        metadata={"role": role, "session_id": session_id}
    )
    memory_vector_store.add_documents([document])


def retrieve_memory(session_id, query, k=1000):
    # Construct a filter string for the session_id
    filter_expression = f"session_id eq '{session_id}'"
    
    # Perform a similarity search with the filter expression
    results = memory_vector_store.similarity_search(
        query=query,
        k=k,
        search_type="similarity",
        filters=filter_expression  # Use the filter expression as a string
    )
    
    # Extract and return the page content of each result
    return [result.page_content for result in results]



In [92]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)


tools = [
    add,
    subtract,
    generate_random_number,
]
llm_with_tools = llm.bind_tools(tools)



system_message = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=[],
        input_types={},
        partial_variables={},
        template="You are a helpful assistant that summarizes data and provides it to the user. You can also send emails, save files to Azure Blob Storage, and perform basic arithmetic operations.",
    ),
    additional_kwargs={},
)

human_message = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["input", "history"],
        input_types={},
        partial_variables={},
        template="{input}\n{history}",
    ),
    additional_kwargs={},
)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        human_message,
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory, save_to_memory=save_to_memory, retrieve_memory=retrieve_memory)

In [93]:
def invoke_agent():
    try:
        data = request.get_json()
        user_prompt = data.get("input")
        history_prompt = data.get("history")
        user_prompt = user_prompt + "\n" + history_prompt
        session_id = data.get("session_id")
        print(session_id)
        if not user_prompt:
            return jsonify({"error": "Missing 'input' parameter in request body"}), 400
        if not session_id:
            return jsonify({"error": "Missing 'session_id' parameter in request body"}), 400

        # Retrieve past memory for the session
        previous_memory = retrieve_memory(session_id, query=user_prompt)

        # Build the input including previous conversation history if available
        input_with_memory = {
            "input": user_prompt,
            "history": previous_memory,
        }
        print(input_with_memory)

        # Invoke the agent
        result = agent_executor.invoke(input_with_memory)

        # Save user and AI response to memory
        save_to_memory(session_id=session_id, role="user", content=user_prompt)
        save_to_memory(session_id=session_id, role="assistant", content=result)

        return jsonify({"result": result})

    except Exception as e:
        logger.error("Error invoking agent: %s", str(e), exc_info=True)
        return jsonify({"error": str(e)}), 500

In [94]:
# Simulate the invoke_agent function to test within Jupyter Notebook
def invoke_agent_notebook(user_prompt, session_id):
    try:
        # Retrieve past memory for the session
        previous_memory = retrieve_memory(session_id, query=user_prompt)
        print(f"Previous memory: {previous_memory}")

        # Build the input including previous conversation history if available
        input_with_memory = {
            "input": user_prompt,
            "history": f"This is the History: {previous_memory}",
        }

        # Invoke the agent
        result = agent_executor.invoke(input_with_memory)

        # Extract the generated response content
        if isinstance(result, dict) and "output" in result:
            assistant_response = result["output"]
        elif isinstance(result, str):
            assistant_response = result
        else:
            raise ValueError("Unexpected format of agent result")

        # Save user and AI response to memory
        save_to_memory(session_id=session_id, role="user", content=user_prompt)
        save_to_memory(session_id=session_id, role="assistant", content=assistant_response)

        return assistant_response

    except Exception as e:
        logger.error("Error invoking agent: %s", str(e), exc_info=True)
        return f"Error: {str(e)}"


In [97]:

# Example usage to test the agent's behavior
session_id = "test_session_3"

# User prompt 1
user_prompt_1 = "My name is Bob"
response_1 = invoke_agent_notebook(user_prompt_1, session_id)
print(f"User: {user_prompt_1}")
print(f"Agent: {response_1}")

# User prompt 2 (testing if it remembers the previous conversation)
user_prompt_2 = "What is my name?"
response_2 = invoke_agent_notebook(user_prompt_2, session_id)
print(f"\nUser: {user_prompt_2}")
print(f"Agent: {response_2}")

ERROR:__main__:Error invoking agent: () Invalid expression: Could not find a property named 'session_id' on type 'search.document'.
Parameter name: $filter
Code: 
Message: Invalid expression: Could not find a property named 'session_id' on type 'search.document'.
Parameter name: $filter
Traceback (most recent call last):
  File "C:\Users\randy\AppData\Local\Temp\ipykernel_30644\3350173261.py", line 5, in invoke_agent_notebook
    previous_memory = retrieve_memory(session_id, query=user_prompt)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\randy\AppData\Local\Temp\ipykernel_30644\3679173744.py", line 20, in retrieve_memory
    results = memory_vector_store.similarity_search(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\randy\Git\stockripper\.venv\Lib\site-packages\langchain_community\vectorstores\azuresearch.py", line 644, in similarity_search
    docs = self.vector_search(query, k=k, **kwargs)
           ^^^^^^^^^^^^^^^^^

User: My name is Bob
Agent: Error: () Invalid expression: Could not find a property named 'session_id' on type 'search.document'.
Parameter name: $filter
Code: 
Message: Invalid expression: Could not find a property named 'session_id' on type 'search.document'.
Parameter name: $filter


ERROR:__main__:Error invoking agent: () Invalid expression: Could not find a property named 'session_id' on type 'search.document'.
Parameter name: $filter
Code: 
Message: Invalid expression: Could not find a property named 'session_id' on type 'search.document'.
Parameter name: $filter
Traceback (most recent call last):
  File "C:\Users\randy\AppData\Local\Temp\ipykernel_30644\3350173261.py", line 5, in invoke_agent_notebook
    previous_memory = retrieve_memory(session_id, query=user_prompt)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\randy\AppData\Local\Temp\ipykernel_30644\3679173744.py", line 20, in retrieve_memory
    results = memory_vector_store.similarity_search(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\randy\Git\stockripper\.venv\Lib\site-packages\langchain_community\vectorstores\azuresearch.py", line 644, in similarity_search
    docs = self.vector_search(query, k=k, **kwargs)
           ^^^^^^^^^^^^^^^^^


User: What is my name?
Agent: Error: () Invalid expression: Could not find a property named 'session_id' on type 'search.document'.
Parameter name: $filter
Code: 
Message: Invalid expression: Could not find a property named 'session_id' on type 'search.document'.
Parameter name: $filter


In [86]:
# User prompt 3 (testing a new query)
user_prompt_3 = "What is my naem?"
response_3 = invoke_agent_notebook(user_prompt_3, session_id)
print(f"\nUser: {user_prompt_3}")
print(f"Agent: {response_3}")

Previous memory: ['What is my name?', 'What is my name?', 'What is my name?', 'What is my name?', 'What is my name?', 'What is my name?', 'What is my name?', 'What did I say my name was?', 'What did I say my name was?', 'What did I say my name was?', 'What did I say my name was?', "I don't have any information about your name. Could you please tell me your name?", 'My name is bob', 'My name is bob', 'My name is bob', 'My name is bob', 'My name is Bob', 'My name is bob!', "It seems I don't have any information about your name. Could you please tell me your name?", "input: What did I say my name was?\noutput: It seems I don't have any information about your name. Could you please tell me your name?", "input: What did I say my name was?\noutput: I don't have any information about your name. Could you please tell me your name?", "input: What is my name?\noutput: I'm sorry, but I don't have access to your name or any personal information unless you provide it to me. How can I assist you tod

In [33]:
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain.memory import VectorStoreRetrieverMemory
from langchain.vectorstores.base import VectorStoreRetriever

# Define the vector store retriever for memory
memory_vector_store = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name="agent-memory",
    embedding_function=embeddings.embed_query,
)

# Create a retriever from the vector store
memory_retriever = VectorStoreRetriever(vectorstore=memory_vector_store)

# Define the memory object using the retriever
memory = VectorStoreRetrieverMemory(
    retriever=memory_retriever,
    memory_key="history",  # This key will be used when referencing memory in prompts
    input_key="input",     # The input key to correlate user prompts
    return_docs=False      # To avoid returning entire documents, useful for simplicity
)


In [34]:
memory

VectorStoreRetrieverMemory(retriever=VectorStoreRetriever(vectorstore=<langchain_community.vectorstores.azuresearch.AzureSearch object at 0x000001CAB89F2090>, search_kwargs={}), input_key='input', exclude_input_keys=())

In [ ]:

# Define the LLM
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

# Create tools and bind them to the LLM
tools = [add, subtract, generate_random_number]
llm_with_tools = llm.bind_tools(tools)

# Define prompts that use the memory explicitly
system_message = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["history"],  # Include memory in the prompt explicitly
        input_types={},
        partial_variables={},
        template="You are a helpful assistant that summarizes data and provides it to the user. "
                 "Here is the conversation history:\n{history}\n"
                 "Please use this history to assist the user effectively.",
    ),
    additional_kwargs={},
)

human_message = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["input"],
        input_types={},
        partial_variables={},
        template="{input}",
    ),
    additional_kwargs={},
)

# Set up the complete prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        human_message,
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Create the agent using the defined prompt
agent = create_tool_calling_agent(llm_with_tools, tools, prompt)

# Pass the agent, tools, and memory to the AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)


In [ ]:

# Function to invoke the agent within a notebook
def invoke_agent_notebook(user_prompt, session_id):
    try:
        # Build the input including memory if available
        input_with_memory = {
            "input": user_prompt
        }

        # Invoke the agent
        result = agent_executor.invoke(input_with_memory)

        # Save user and AI response to memory explicitly
        save_to_memory(session_id=session_id, role="user", content=user_prompt)
        save_to_memory(session_id=session_id, role="assistant", content=result)

        return result

    except Exception as e:
        logger.error("Error invoking agent: %s", str(e), exc_info=True)
        return f"Error: {str(e)}"

# Example usage to test the agent's behavior
session_id = "test_session_1"

# User prompt 1
user_prompt_1 = "What is an aggressive investment strategy?"
response_1 = invoke_agent_notebook(user_prompt_1, session_id)
print(f"User: {user_prompt_1}")
print(f"Agent: {response_1}")

# User prompt 2 (testing if it remembers the previous conversation)
user_prompt_2 = "Can you tell me more about the risks involved in aggressive investments?"
response_2 = invoke_agent_notebook(user_prompt_2, session_id)
print(f"\nUser: {user_prompt_2}")
print(f"Agent: {response_2}")

# User prompt 3 (testing memory recall)
user_prompt_3 = "What did I last ask?"
response_3 = invoke_agent_notebook(user_prompt_3, session_id)
print(f"\nUser: {user_prompt_3}")
print(f"Agent: {response_3}")


In [30]:
# User prompt 3 (testing a new query)
user_prompt_3 = "What did I last ask?"
response_3 = invoke_agent_notebook(user_prompt_3, session_id)
print(f"\nUser: {user_prompt_3}")
print(f"Agent: {response_3}")



> Entering new AgentExecutor chain...
I'm unable to access previous interactions or conversations. Could you please remind me what you last asked?

> Finished chain.

User: What did I last ask?
Agent: I'm unable to access previous interactions or conversations. Could you please remind me what you last asked?


In [27]:
session_id = "test_session_1"

# User prompt 1
user_prompt_1 = "What is an aggressive investment strategy?"
response_1 = invoke_agent_notebook(user_prompt_1, session_id)
print(f"User: {user_prompt_1}")
print(f"Agent: {response_1}")

# User prompt 2 (testing if it remembers the previous conversation)
user_prompt_2 = "Can you tell me more about the risks involved in aggressive investments?"
response_2 = invoke_agent_notebook(user_prompt_2, session_id)
print(f"\nUser: {user_prompt_2}")
print(f"Agent: {response_2}")



> Entering new AgentExecutor chain...
An aggressive investment strategy is an approach to investing that seeks to achieve high returns by taking on a higher level of risk. This strategy typically involves investing in assets that have the potential for significant price appreciation, but also come with a greater chance of loss. Here are some key characteristics of an aggressive investment strategy:

1. **High Risk, High Reward**: Investors aim for substantial returns, often targeting stocks, options, or other financial instruments that can provide rapid growth.

2. **Focus on Growth Stocks**: Investments are often concentrated in growth stocks, which are expected to grow at an above-average rate compared to other companies.

3. **Market Timing**: Aggressive investors may actively trade to take advantage of market fluctuations, often employing technical analysis to time their trades.

4. **Concentration in Few Investments**: Portfolios may be less diversified, focusing on a limited nu

ERROR:__main__:Error invoking agent: 1 validation error for Document
page_content
  Input should be a valid string [type=string_type, input_value={'input': 'What is an agg...tand potential losses.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
Traceback (most recent call last):
  File "C:\Users\randy\AppData\Local\Temp\ipykernel_30644\584253365.py", line 17, in invoke_agent_notebook
    save_to_memory(session_id=session_id, role="assistant", content=result)
  File "C:\Users\randy\AppData\Local\Temp\ipykernel_30644\61312367.py", line 7, in save_to_memory
    document = Document(
               ^^^^^^^^^
  File "c:\Users\randy\Git\stockripper\.venv\Lib\site-packages\langchain_core\documents\base.py", line 285, in __init__
    super().__init__(page_content=page_content, **kwargs)  # type: ignore[call-arg]
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\randy\Git\stockripper\.venv\Lib\site-packages\langchain_co

User: What is an aggressive investment strategy?
Agent: Error: 1 validation error for Document
page_content
  Input should be a valid string [type=string_type, input_value={'input': 'What is an agg...tand potential losses.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type


> Entering new AgentExecutor chain...
Aggressive investments typically involve a higher level of risk compared to more conservative investment strategies. Here are some key risks associated with aggressive investments:

1. **Market Volatility**: Aggressive investments often include stocks of small-cap companies or sectors that are more prone to market fluctuations. This can lead to significant price swings, which can result in substantial gains or losses.

2. **Loss of Capital**: Due to their high-risk nature, aggressive investments can lead to considerable losses. Investors may lose a significant portion or even all of their invested capital, especially if the marke

ERROR:__main__:Error invoking agent: 1 validation error for Document
page_content
  Input should be a valid string [type=string_type, input_value={'input': 'Can you tell m...e some of these risks."}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
Traceback (most recent call last):
  File "C:\Users\randy\AppData\Local\Temp\ipykernel_30644\584253365.py", line 17, in invoke_agent_notebook
    save_to_memory(session_id=session_id, role="assistant", content=result)
  File "C:\Users\randy\AppData\Local\Temp\ipykernel_30644\61312367.py", line 7, in save_to_memory
    document = Document(
               ^^^^^^^^^
  File "c:\Users\randy\Git\stockripper\.venv\Lib\site-packages\langchain_core\documents\base.py", line 285, in __init__
    super().__init__(page_content=page_content, **kwargs)  # type: ignore[call-arg]
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\randy\Git\stockripper\.venv\Lib\site-packages\langchain_co


User: Can you tell me more about the risks involved in aggressive investments?
Agent: Error: 1 validation error for Document
page_content
  Input should be a valid string [type=string_type, input_value={'input': 'Can you tell m...e some of these risks."}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type


In [ ]:

llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)


tools = [
    add,
    subtract,
    generate_random_number,
]
llm_with_tools = llm.bind_tools(tools)



system_message = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=[],
        input_types={},
        partial_variables={},
        template="You are a helpful assistant that summarizes data and provides it to the user. You can also send emails, save files to Azure Blob Storage, and perform basic arithmetic operations.",
    ),
    additional_kwargs={},
)

human_message = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["input"],
        input_types={},
        partial_variables={},
        template="{input}",
    ),
    additional_kwargs={},
)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        human_message,
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)


@app.route("/agents/mailworker", methods=["POST"])
def invoke_agent():
    try:
        data = request.get_json()
        user_prompt = data.get("input")
        session_id = data.get("session_id")
        if not user_prompt:
            return jsonify({"error": "Missing 'input' parameter in request body"}), 400
        if not session_id:
            return (
                jsonify({"error": "Missing 'session_id' parameter in request body"}),
            )

        result = agent_executor.invoke({"input": user_prompt})
        return jsonify({"result": result})
    except Exception as e:
        logger.error("Error invoking agent: %s", str(e), exc_info=True)
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    # Run the Flask app on port 5000
    app.run(host="0.0.0.0", port=5000, debug=True)
